In [1]:
import torch
import torch.nn as nn

from src.data.adversarial import AdversarialDataProvider
from src.data.reconstruction import ReconstructionDataProvider
from src.models.stylist import Stylist
from src.models.generator import Generator
from src.models.discriminator import Discriminator
from src.render.mesh_renderer import MeshPointsRenderer

class Trainer:
    
    def __init__(self, config):
        self.G_noise_amp =  config.G_noise_amp
        self.z_size = config.latent_size
        
        self.device = torch.device("cuda")
        
        # Data providers
        self.RDP = ReconstructionDataProvider(config)
        self.ADP = AdversarialDataProvider(config)
        
        self.S = Stylist(config).to(self.device)
        self.G = Generator(config).to(self.device)
        self.D = Discriminator(config).to(self.device)
        self.R = MeshPointsRenderer(config).to(self.device)
        self.R.setup(self.device)
        
        # Optimizers
        self.optim_G = torch.optim.Adam(self.G.parameters(), lr=0.0001)
        self.optim_S = torch.optim.Adam(self.S.parameters(), lr=0.0001)
        self.optim_D = torch.optim.Adam(self.D.parameters(), lr=0.0001)
        # Loss functions
        self.adversarial_loss = nn.BCELoss()
        self.reconstruction_loss = nn.L1Loss()
        
    def step(self):
        pass
        
    
    def adversarial_step(self):
        batch, mean_std = self.ADP.next_batch(labels=True, device=self.device)
        images, points = batch['large'], batch['points'] # ?????
        label_real = batch['label_real']
        label_fake = batch['label_fake']
        
        # (1) Update Discriminator
        ## Train with all-real batch
        self.optim_D.zero_grad()
        output = self.D(images)        
        errD_real =  self.adversarial_loss(output, label_real)
        errD_real.backward()
        D_x = output.mean().item() 
        
        ## Train with all-fake batch
        style = self.S(images)
        pts =  self.G(points, style)
        g_images = self.R(pts, mean_std=mean_std)                
        output = self.D(g_images.detach())  
        errD_fake = self.adversarial_loss(output, label_fake)    
        errD_fake.backward()
        
        self.optim_D.step()
        
        # (2) Update Generator and Stylist
        self.optim_S.zero_grad()
        self.optim_G.zero_grad()
        
        output = self.D(g_images)    
        errG = self.adversarial_loss(output, label_real)
        errG.backward()
        
        self.optim_S.step()
        self.optim_G.step()
        
        return {
            'errD_real': errD_real.item(),
            'errD_fake': errD_fake.item(),
            'errD': (errD_real + errD_fake).item(),
            'errG': errD_fake.item(),
        }
        
    def reconstruction_step(self):
        batch = self.RDP.next_batch(self.device)        
        pts_fine = batch['points']
        bs = pts_fine.size(0)
        
        pts_noise = pts_fine + torch.randn_like(pts_fine) * self.G_noise_amp
        style = torch.zeros(bs, self.z_size, device=self.device)
        
        self.optim_G.zero_grad()
        
        vertices = self.G(pts_noise, style)
        errG_rec = self.reconstruction_loss(vertices, pts_fine)
        
        errG_rec.backward()
        
        self.optim_G.step()
        return {
            'errG_rec': errG_rec.item(),
        }


from src.config import get_parser

config = get_parser().parse_args(args=[])    
trainer = Trainer(config)
trainer

torch.Size([16, 3, 512, 512])


In [4]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [5]:
import random
random.random()

0.9127995548808574

In [6]:
epochs = 100
steps = len(trainer.RDP) + len(trainer.ADP)

threshold = len(trainer.RDP) / len(trainer.ADP)

for epoch_no in range(epochs):
    for step_no in range(steps):
        if random.random() < 0.5: #threshold:
            losses = trainer.reconstruction_step()
        else:
            losses = trainer.adversarial_step()
        for key in losses.keys():
            writer.add_scalar(f'Loss/{key}', losses[key], epoch_no * steps + step_no)
    print(epoch_no)

/home/bobi/miniconda3/envs/pytorch3d/lib/python3.8/site-packages/pytorch3d/transforms/transform3d.py:726: UserWarning:

R is not a valid rotation matrix



KeyboardInterrupt: 

In [2]:
trainer.reconstruction_step()

{'errG_rec': 0.26973843574523926}